<a href="https://colab.research.google.com/github/1moi6/pyfuzzy-toolbox/blob/main/notebooks_colab/Aula_2/04_atividade_predicao_voto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade Prática: Sistema de Predição de Voto

## 📚 Contexto

Vamos criar um sistema de inferência fuzzy para prever a **chance de um eleitor votar** em um candidato, baseado em:
- **Renda Familiar** (0 a 30 mil R$)
- **Escolaridade** (0 a 15 anos de estudo)

Este é um exemplo didático para praticar sistemas fuzzy com múltiplas entradas!

---

## 🔧 Instalação

In [ ]:
!pip install pyfuzzy-toolbox -q

print('✅ pyfuzzy-toolbox instalado!')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import fuzzy_systems as fs
from fuzzy_systems import MamdaniSystem

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

print('✅ Bibliotecas carregadas!')
print(f'   pyfuzzy-toolbox: {fs.__version__}')

## Passo 1: Criar Sistema e Definir Variáveis

### Renda Familiar (0 a 30 mil R$)
- **Baixa**: trapezoidal [0, 0, 5, 15]
- **Média**: trapezoidal [5, 15, 20, 25]
- **Alta**: trapezoidal [20, 25, 30, 30]

### Escolaridade (0 a 15 anos)
- **Baixa**: trapezoidal [0, 0, 3, 6]
- **Média**: triangular [4, 7.5, 11]
- **Alta**: trapezoidal [9, 12, 15, 15]

In [ ]:
# ============================================================================
# Criar Sistema Mamdani
# ============================================================================

sistema = MamdaniSystem()

# ============================================================================
# Variável 1: Renda Familiar
# ============================================================================

sistema.add_input('renda', (0, 30))

sistema.add_term('renda', 'baixa', 'trapezoidal', (0, 0, 5, 15))
sistema.add_term('renda', 'media', 'trapezoidal', (5, 15, 20, 25))
sistema.add_term('renda', 'alta', 'trapezoidal', (20, 25, 30, 30))

# ============================================================================
# Variável 2: Escolaridade
# ============================================================================

sistema.add_input('escolaridade', (0, 15))

sistema.add_term('escolaridade', 'baixa', 'trapezoidal', (0, 0, 3, 6))
sistema.add_term('escolaridade', 'media', 'triangular', (4, 7.5, 11))
sistema.add_term('escolaridade', 'alta', 'trapezoidal', (9, 12, 15, 15))

# ============================================================================
# Saída: Chance de Votar (0 a 100%)
# ============================================================================

sistema.add_output('chance_voto', (0, 100))

sistema.add_term('chance_voto', 'baixa', 'triangular', (0, 0, 50))
sistema.add_term('chance_voto', 'media', 'triangular', (25, 50, 75))
sistema.add_term('chance_voto', 'alta', 'triangular', (50, 100, 100))

print('✅ Variáveis definidas!')
print('   • Renda: baixa, media, alta')
print('   • Escolaridade: baixa, media, alta')
print('   • Chance de voto: baixa, media, alta')

## Visualizar Funções de Pertinência

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 12))

# Renda
x_renda = np.linspace(0, 30, 200)
for term_name, term in sistema.inputs['renda'].terms.items():
    mu = term.membership(x_renda)
    axes[0].plot(x_renda, mu, linewidth=2, label=term_name.capitalize())
    axes[0].fill_between(x_renda, 0, mu, alpha=0.2)

axes[0].set_title('Renda Familiar (mil R$)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Renda (mil R$)')
axes[0].set_ylabel('Grau de Pertinência')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([-0.05, 1.05])

# Escolaridade
x_esc = np.linspace(0, 15, 200)
for term_name, term in sistema.inputs['escolaridade'].terms.items():
    mu = term.membership(x_esc)
    axes[1].plot(x_esc, mu, linewidth=2, label=term_name.capitalize())
    axes[1].fill_between(x_esc, 0, mu, alpha=0.2)

axes[1].set_title('Escolaridade (anos)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Anos de Estudo')
axes[1].set_ylabel('Grau de Pertinência')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([-0.05, 1.05])

# Chance de Voto
x_chance = np.linspace(0, 100, 200)
for term_name, term in sistema.outputs['chance_voto'].terms.items():
    mu = term.membership(x_chance)
    axes[2].plot(x_chance, mu, linewidth=2, label=term_name.capitalize())
    axes[2].fill_between(x_chance, 0, mu, alpha=0.2)

axes[2].set_title('Chance de Votar (%)', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Chance (%)')
axes[2].set_ylabel('Grau de Pertinência')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
axes[2].set_ylim([-0.05, 1.05])

plt.tight_layout()
plt.show()

print('✅ Funções visualizadas!')

## Passo 2: Criar Base de Regras

### 📜 Regras (Exemplo fictício para fins didáticos):

1. **SE** renda é ALTA **E** escolaridade é ALTA **ENTÃO** chance é BAIXA
2. **SE** renda é ALTA **E** escolaridade é MÉDIA **ENTÃO** chance é BAIXA  
3. **SE** renda é MÉDIA **E** escolaridade é MÉDIA **ENTÃO** chance é MÉDIA
4. **SE** renda é BAIXA **E** escolaridade é BAIXA **ENTÃO** chance é ALTA
5. **SE** renda é BAIXA **E** escolaridade é MÉDIA **ENTÃO** chance é ALTA

**Nota**: Este é um exemplo didático. Em aplicações reais, as regras devem ser baseadas em dados ou conhecimento especialista.

In [ ]:
# ============================================================================
# Definir Regras
# ============================================================================

sistema.add_rule(
    antecedents={'renda': 'alta', 'escolaridade': 'alta'},
    consequent={'chance_voto': 'baixa'}
)

sistema.add_rule(
    antecedents={'renda': 'alta', 'escolaridade': 'media'},
    consequent={'chance_voto': 'baixa'}
)

sistema.add_rule(
    antecedents={'renda': 'media', 'escolaridade': 'media'},
    consequent={'chance_voto': 'media'}
)

sistema.add_rule(
    antecedents={'renda': 'baixa', 'escolaridade': 'baixa'},
    consequent={'chance_voto': 'alta'}
)

sistema.add_rule(
    antecedents={'renda': 'baixa', 'escolaridade': 'media'},
    consequent={'chance_voto': 'alta'}
)

print('📜 Base de Regras:')
print('   R1: SE renda ALTA E escolaridade ALTA → chance BAIXA')
print('   R2: SE renda ALTA E escolaridade MÉDIA → chance BAIXA')
print('   R3: SE renda MÉDIA E escolaridade MÉDIA → chance MÉDIA')
print('   R4: SE renda BAIXA E escolaridade BAIXA → chance ALTA')
print('   R5: SE renda BAIXA E escolaridade MÉDIA → chance ALTA')
print(f'\n✅ {len(sistema.rules)} regras criadas!')

## Passo 3: Testar o Sistema

In [ ]:
# ============================================================================
# Caso de Teste: Renda=22, Escolaridade=12
# ============================================================================

print('='*70)
print('🧪 TESTE: Renda=22 mil, Escolaridade=12 anos')
print('='*70)

resultado = sistema.evaluate({
    'renda': 22,
    'escolaridade': 12
})

chance = resultado['chance_voto']

print(f'\n📊 Resultado: Chance de votar = {chance:.2f}%')

# Mostrar fuzzificação
graus_renda = sistema.inputs['renda'].fuzzify(22)
graus_esc = sistema.inputs['escolaridade'].fuzzify(12)

print(f'\nGraus de pertinência (Renda=22):')
for termo, mu in graus_renda.items():
    print(f'  {termo}: {mu:.3f}')

print(f'\nGraus de pertinência (Escolaridade=12):')
for termo, mu in graus_esc.items():
    print(f'  {termo}: {mu:.3f}')

print('='*70)

## Testar Múltiplos Cenários

In [ ]:
print('='*80)
print('🧪 TESTANDO MÚLTIPLOS CENÁRIOS')
print('='*80)
print()

cenarios = [
    (5, 3, 'Baixa renda, baixa escolaridade'),
    (15, 8, 'Renda média, escolaridade média'),
    (25, 12, 'Alta renda, alta escolaridade'),
    (10, 12, 'Renda média, alta escolaridade'),
    (28, 5, 'Alta renda, baixa escolaridade'),
]

print(f'{"Renda":<8} {"Escol":<8} {"Chance (%)":<12} {"Cenário"}')
print('-'*80)

for r, e, descricao in cenarios:
    resultado = sistema.evaluate({
        'renda': r,
        'escolaridade': e
    })
    c = resultado['chance_voto']
    print(f'{r:<8} {e:<8} {c:<12.2f} {descricao}')

print('\n' + '='*80)
print('✅ Testes concluídos!')

## Superfície de Controle 3D

In [ ]:
print('🔄 Gerando superfície de controle...')

# Criar grid
renda_range = np.linspace(0, 30, 30)
esc_range = np.linspace(0, 15, 30)
RENDA, ESC = np.meshgrid(renda_range, esc_range)

# Calcular chance para cada ponto
CHANCE = np.zeros_like(RENDA)
for i in range(RENDA.shape[0]):
    for j in range(RENDA.shape[1]):
        try:
            resultado = sistema.evaluate({
                'renda': RENDA[i,j],
                'escolaridade': ESC[i,j]
            })
            CHANCE[i,j] = resultado['chance_voto']
        except:
            CHANCE[i,j] = np.nan

# Plotar
fig = plt.figure(figsize=(16, 7))

ax1 = fig.add_subplot(121, projection='3d')
surf = ax1.plot_surface(RENDA, ESC, CHANCE, cmap='RdYlGn', alpha=0.9)
ax1.set_xlabel('Renda Familiar (mil R$)', fontsize=11)
ax1.set_ylabel('Escolaridade (anos)', fontsize=11)
ax1.set_zlabel('Chance de Votar (%)', fontsize=11)
ax1.set_title('Superfície de Controle 3D', fontsize=13, fontweight='bold')
ax1.view_init(elev=20, azim=135)
fig.colorbar(surf, ax=ax1, shrink=0.5, label='Chance (%)')

ax2 = fig.add_subplot(122)
contour = ax2.contourf(RENDA, ESC, CHANCE, levels=15, cmap='RdYlGn')
ax2.set_xlabel('Renda Familiar (mil R$)', fontsize=12)
ax2.set_ylabel('Escolaridade (anos)', fontsize=12)
ax2.set_title('Mapa de Contorno', fontsize=13, fontweight='bold')
fig.colorbar(contour, ax=ax2, label='Chance (%)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('✅ Superfície gerada!')
print('\n💡 Cores verdes = maior chance de votar')
print('   Cores vermelhas = menor chance de votar')

---

## 🎓 Conclusão

Parabéns! Você criou um sistema fuzzy completo para predição de voto usando **pyfuzzy-toolbox**!

### ✅ O que você praticou:

1. Criar sistema com **múltiplas entradas**
2. Definir variáveis com **diferentes funções** de pertinência
3. Criar **base de regras** complexa
4. **Testar** sistema com múltiplos cenários
5. Gerar e interpretar **superfícies de controle 3D**
6. Usar a biblioteca **pyfuzzy-toolbox** de forma eficiente

### 🔑 Pontos-Chave:

- Sistemas fuzzy podem ter **múltiplas entradas** facilmente
- Regras combinam variáveis com operadores **AND/OR**
- Superfície 3D mostra **comportamento global** do sistema
- **pyfuzzy-toolbox** simplifica drasticamente a implementação
- Com apenas algumas linhas criamos um sistema completo!

### 🚀 Desafios Extras:

1. **Modifique as regras** e observe como muda o resultado
2. **Adicione uma terceira entrada** (ex: idade do eleitor)
3. **Teste diferentes funções** de pertinência
4. **Compare com Sugeno** - implemente o mesmo sistema
5. **Ajuste os parâmetros** para melhorar o modelo

### 💡 Aplicações Reais:

Este tipo de sistema fuzzy pode ser usado em:
- Análise de preferências eleitorais
- Sistemas de recomendação
- Análise de risco de crédito
- Sistemas de decisão multicritério
- Classificação de clientes

### 📚 Recursos:

- **PyPI**: https://pypi.org/project/pyfuzzy-toolbox/
- **GitHub**: https://github.com/1moi6/pyfuzzy-toolbox
- **Documentação**: https://github.com/1moi6/pyfuzzy-toolbox#readme

---

*Atividade desenvolvida para o Minicurso de Sistemas de Inferência Fuzzy - 2025*  
*Usando pyfuzzy-toolbox v1.0.0*